In [1]:
from utils import * 

best_model = base.sort_values('eval/f1', ascending=False).iloc[0]
tokenizer, base_model, pretrained_model = get_model(best_model.ID)

base_model = base_model
pretrained_model = pretrained_model

/Users/henrywilliams/Documents/programming/python/ai/malbert-test/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:   7 of 7 files downloaded.  
wandb:   7 of 7 files downloaded.  


In [2]:
A = get_attention(base_model, dataset['test'][0], tokenizer)

RobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [5]:
A[0].shape

torch.Size([916, 2, 32, 32])

In [8]:
import torch

def attention_attribution(F, A: torch.Tensor, A_h_index: int, m: int) -> torch.Tensor:
    A_h = A[A_h_index]
    total_grad = torch.zeros_like(A_h)

    for k in range(1, m + 1):
        alpha = k / m
        Ak = A * alpha
        print(Ak.shape)
        output = F(Ak)  # output must be scalar or batched scalar per head
        grad = torch.autograd.grad(
            outputs=output,
            inputs=Ak,
            retain_graph=True,
            create_graph=False,
            allow_unused=True
        )[0]
        if grad is not None:
            total_grad += grad[A_h_index]

    return (A_h / m) * total_grad

attention_attribution(base_model, A[-1], 0, 20)

torch.Size([916, 2, 32, 32])

ValueError: too many values to unpack (expected 2)